In [49]:
# YYYY - MM - DD
start_date = "2020-06-01"
end_date = "2020-07-15"

user_id = "" # leave blank if you want all records

protocol_num = 1 # 1 for mosquito, 2 for land cover, 3 for trees, 4 for cloud

download_pictures = False

In [50]:
import requests
import json

protocols = {1 : "mosquito_habitat_mapper", 2 : "land_covers", 3 : "tree_heights", 4 : "sky_conditions"}
protocol = protocols[protocol_num]
if user_id:
    url = f"https://api.globe.gov/search/v1/measurement/protocol/measureddate/userid/?protocols={protocol}&startdate={start_date}&enddate={end_date}&userid={user_id}&geojson=TRUE&sample=FALSE"
else:
    url = f"https://api.globe.gov/search/v1/measurement/protocol/measureddate/?protocols={protocol}&startdate={start_date}&enddate={end_date}&geojson=TRUE&sample=FALSE"
response = requests.get(url)
data = response.json()
protocol_name = protocol.replace('_', '')

data["features"] = [
    feature for feature in data["features"] if feature["properties"][f"{protocol_name}DataSource"] == "GLOBE Observer App" 
]

for feature in data["features"]:
    
    try:
        
        feature["geometry"]["coordinates"][0] = feature["properties"][f"{protocol_name}MeasurementLongitude"]
        feature["geometry"]["coordinates"][1] = feature["properties"][f"{protocol_name}MeasurementLatitude"]
    except KeyError:
        print("skipping") # some entries don't have measured at values

# write to file
with open(f'{protocol}-{user_id}_measuredAt.json', 'w') as fp:
    json.dump(data, fp)

In [51]:
import os

def get_picture(picture_url, file_name):
    downloaded_obj = requests.get(picture_url, allow_redirects=True)
    parent_dir = os.path.join("downloaded_images", file_name)
    with open(parent_dir, "wb") as file:
        file.write(downloaded_obj.content)
def download_picture(picture_url):
    if "https://" in url:
        photo_id = re.search(r'(?<=\d\d\d\d\/\d\d\/\d\d\/).*(?=\/)', url).group(0)
        file_name = f"{name}-{photo_id}-{protocol_name}.jpg".replace(":","-")
        get_picture(url, file_name)
    

In [52]:
import re
directions = ["DownwardPhotoUrl", "EastPhotoUrl", "NorthPhotoUrl", "SouthPhotoUrl", "WestPhotoUrl", "UpwardPhotoUrl"]

sort_user = True
for feature in [data["features"][0]]:
    name = f"{feature['properties'][f'{protocol_name}MeasuredAt']}"
    if sort_user:
        name = f"{feature['properties'][f'{protocol_name}Userid']}-{name}"
    else:
        name = f"{feature['properties']['siteName']}-{name}"
    
    if protocol_num == 2 or protocol_num == 4:
        for direction in directions:
            url = feature["properties"][f"{protocol_name}{direction}"]
            download_picture(url)
    else:
        if protocol_num == 1:
            urls = feature["properties"][f"{protocol_name}WaterSourcePhotoUrls"]
        else:
            urls = feature["properties"][f"{protocol_name}TreePhotoUrls"]
        urls = urls.split(";")
        for url in urls:
            download_picture(url)
            
            
    
        

In [54]:
import shutil

shutil.make_archive("downloaded_pictures", "zip", "downloaded_images")
shutil.rmtree("downloaded_images")
os.mkdir("downloaded_images")


'C:\\Users\\piphi\\Documents\\SEES GeoJSON Converter\\downloaded_pictures.zip'